# MATH 3375 Examples Notebook #22

# Simple Simulations

Using known probability distributions, we first demonstrate a simulation as a way to model expected sales at a coffee shop. 

## Scenario
The arrival rate of customers to a hotel coffee shop between 6:00 and 9:00 a.m. follows a Poisson distribution with an average of 8 customers arriving every 5 minutes.  Thus, measuring time in 1-minute intervals, $\lambda = \frac{8}{5}$. 

Now suppose that the amount spent by each customer follows a normal distribution with a mean of $6.82 and a standard deviation of 98 cents.

## Questions

1) What is the projected average total sales at the coffee shop on a given day from 6:00 to 9:00 a.m.?

2) What is a 95% confidence interval for average total sales at the coffee shop during these hours?

3) What is the probability the total sales during these hours on a given day will be $2000 or more?

### Step 1. Set up to use designated probability distributions

In [ ]:
# Set up parameters for known distributions
lambda <- 8/5
mean_spend <- 6.82
sd_spend <- 0.98

### Step 2. Generate customers

First we will simply use the arrival distribution (Poisson with given $\lambda$) to generate the number of customers arriving each minute during the 3-hour interval. Note that this will be a total of 180 minutes.

In [ ]:
customers <- rpois(180,lambda)
print(customers)

### Step 3. Generate amount spent per customer

We will use the specified spending distribution (Normal with given $\mu$ and $\sigma$) to generate an amount spent by EACH customer for each of the 180 minutes.

Notice that in any minute, there can be multiple customers; the variable **c** represents the number of customers that arrive in that minute. Thus, the line 

    if (c) 

will be TRUE if c is non-zero, and FALSE when c is zero (no customers during that minute). So we only calculate **spent** if there were customers. Also note that **spent** is a vector of multiple amounts-- one for each customer.

In [ ]:
customers <- rpois(180,lambda)
for (c in customers) {
    if (c) {
        spent <- round(rnorm(c,mean=mean_spend,sd=sd_spend),2)
        #print(spent)
        }
}

### Step 4. Compute total amount spent 

Update the above code to keep a running total of the amount spent by each customer.  Since **spent** represents amounts spent by multiple customers in a given minute, we use **sum** to combine them, then add that sum to the total.



In [ ]:
total <- 0

customers <- rpois(180,lambda)
for (c in customers) {
    if (c) {
        spent <- round(rnorm(c,mean=mean_spend,sd=sd_spend),2)
        total <- total + sum(spent)
        }
}
cat("Total spent: ", total)

### Step 5. Repetition

Above is a single simulation run that gives a value for total spent during the 3-hour period in question.  Is this a good answer to our first question (projected average total sales)?  

Run the cell above repeatedly, and you will see many different answers, due to the inherent randomness of generating random values from a distribution. You should notice a lot of variability in the final totals. 

To get a better estimate of the average total sales, we should run that process many times and use the average of all the results as a much more stable estimate. The code below places the above simulation in a **_loop_** that runs 10,000 times. The **results** vector is used to store the result (total spent) for each of the 10,000 simulations.

In [ ]:
results <- c()

for (sim in 1:10000) {
    total <- 0

    customers <- rpois(180,lambda)
    for (c in customers) {
        if (c) {
            spent <- round(rnorm(c,mean=mean_spend,sd=sd_spend),2)
            total <- total + sum(spent)
            }
    }
    results[sim] <- total
}

head(results)



#### Examine the distribution of simulation results

It's useful to look at the distribution of simulation results, both visually and numerically. This is done below. Recall that each result represents the total spent in one simulation.  

In [ ]:
hist(results)

summary(results)

### Step 6. Use results to answer questions.

##### Q1: What is the projected average total sales at the coffee shop on a given day from 6:00 to 9:00 a.m.?

Based on the data summary above, the mean could be reported as a reasonable estimate of average total sales.

##### Q2: What is a 95% confidence interval for average total sales at the coffee shop during these hours?

The middle 95% of the results shown above can be used to create a 95% confidence interval for average total sales. The process is:

* Put results in order
* Find the **_position_** of the lowest and highest values in the interval.
* Use the values in the low/high positions as the lower and upper bounds of the interval.

The middle 95% has boundaries to drop the lowest 2.5% and the highest 2.5% of all results. For our 10,000 results, this means we should drop the lowest 250, using 251 for the position of our lower boundary. Then we should drop the highest 250, using 9750 as the position of our upper boundary.

This process is carried out below.  

In [ ]:
#Put results in order
results <- sort(results)   

#Find position of lower and upper boundaries
lower <- round(.025*length(results)) 
upper <- length(results) - lower

#Use results at boundary positions
CI_lower <- results[lower+1]
CI_upper <- results[upper]

paste0("95% Confidence Interval: [",CI_lower,", ",CI_upper,"]")

##### Q3: What is the probability the total sales during these hours on a given day will be $2000 or more?

An empirical probability can be computed by counting the number of results with a total of 2000 or more, and then dividing by the total number of results.

In [ ]:
sum(as.integer(results>=2000))

In [ ]:
sum(as.integer(results>=2000))/length(results)

## Identifying the right distribution

The above simulations used specific distributions, but where did these come from? Typically some data collection is performed first and these data are used to identify an appropriate distribution. We show this process below, using sample data from recent sales of the coffee shop.

In [ ]:
spend_data <- read.csv("coffee_spend.csv")
head(spend_data)

First, we look at the sample statistics and plots to visualize the potential distribution.

In [ ]:
hist(spend_data$spent)

In [ ]:
summary(spend_data$spent)
sd(spend_data$spent)

### Interpretation

The histogram suggests that the normal distribution is **_possibly_** a good fit, and the sample statistics suggest that the distribution would have mean about 6.84 and standard deviation of about 0.99. 

The plot below can confirm whether the data fit the normal distribution, but it cannot confirm the parameters (mean, sd) of that distribution.  Notice that the theoretical quantiles match the **_standard_** normal distribution (mean=0, sd=1).

In [ ]:
qqnorm(spend_data$spent)
qqline(spend_data$spent)

### Create your own QQ Plot for ANY distribution

Suppose we want to test our data against a specific distribution with parameters we have defined. Below, we compare our data to a Normal distribution with mean of 6.50 and standard deviation of 0.80.  (This could be the last estimate recorded from several months ago.)

In [ ]:
qqplot(qnorm(seq(0,1,by=.01),mean=6.50,sd=0.80),spend_data$spent,xlab="Theoretical Quantiles")
abline(a=0,b=1)

#### Interpretation

We can see that the $N(6.50,0.80)$ distribution is not a good fit for the data we collected.  However, using the sample values to estimate a $N(6.84,0.99)$ distribution, we can see below that it is a good fit for the data.

In [ ]:
qqplot(qnorm(seq(0,1,by=.01),mean=6.84,sd=0.99),spend_data$spent,xlab="Theoretical Quantiles")
abline(a=0,b=1)

### Testing other distributions

Note that the QQ plot is not reserved for normal distributions.  Below we show that the exponential distribution with $\lambda=0.25$ is also not a good fit for our data.

In [ ]:
qqplot(qexp(seq(0,1,by=.01),rate=0.25),spend_data$spent,xlab="Theoretical Quantiles")
abline(a=0,b=1)